# **calt Minimal Demo**

#### Installation

In [ ]:
%%capture
%pip install calt-x

#### Dataset generation

Task: Polynomial addition

In [ ]:
# import PolynomialSampler
import random
from calt.dataset_generator.sympy import PolynomialSampler
# from calt.dataset_generator.sage import PolynomialSampler  

def sum_problem_generator(seed):
    random.seed(seed)

    # Initialize polynomial sampler
    sampler = PolynomialSampler(
        symbols="x0, x1",
        field_str="GF(7)",
        max_num_terms=2,
        max_degree=2,
        min_degree=1,
    )

    # Generate problem polynomials using sampler
    F = sampler.sample(num_samples=2)

    # Generate solution polynomial g (sum of F)
    g = sum(F)

    return F, g

In [ ]:
from calt.generator.sympy import DatasetGenerator
# from calt.generator.sage import DatasetGenerator

dataset_generator = DatasetGenerator(n_jobs=12, root_seed=100)  # use 12 cores for parallel processing

dataset_generator.run(
    dataset_sizes={"train": 10000, "test": 1000},
    problem_generator=sum_problem_generator,
    save_dir='.',
)

#### Model Training

In [ ]:
import yaml

from calt.data_loader.data_loader import DataPipeline
from calt.model.transformer import ModelPipeline
from calt.trainer.trainer import TrainPipeline

# load configs 
with open("examples/demos/configs/config.yaml", "r") as f:
    config = yaml.load(f)
    
# set up pipelines
data_pipeline = DataPipeline(config=config.io) # load and proccess data for input
model = ModelPipeline(config=config.model) # build a Transformer model
trainer = TrainPipeline(config=config.train) # set up training pipeline

# train and evaluate
metrics = trainer.train_and_eval()

print(f'success rate on test set: {100*metrics["success_rate"]:.1f} %')

### Showcasing

In [ ]:
from calt.utils.demo import showcase

dataset = data_pipeline.test_dataset
showcase(dataset, success_cases=True, num_show=5)
showcase(dataset, success_cases=False, num_show=5)